In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
#The os.chdir() function in Python changes the current working
#directory for the duration of the Python script's execution.
os.chdir('drive/MyDrive/UCLA_WiDS_Team 2/DATA/TRAIN')

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train_df = pd.read_csv('train_data.csv')
train_df

In [ ]:
print(os.getcwd())
os.chdir('../TEST')
print(os.getcwd())

In [ ]:
test_df = pd.read_csv('test_data.csv')
test_df

In [ ]:
adhd = train_df['ADHD_Outcome']
sex = train_df['Sex_F']

In [ ]:
num_adhd = train_df['ADHD_Outcome'].value_counts()
num_adhd    #noticed the values are imbalanced

In [ ]:
num_sex = train_df['Sex_F'].value_counts()
num_sex     #noticed the values are imbalanced

In [ ]:
def drop_uncommon_columns(df1, df2):
    # Find common columns
    common_columns = df1.columns.intersection(df2.columns)

    # Keep only common columns
    df1 = df1[common_columns]
    df2 = df2[common_columns]

    return df1, df2

train_df, test_df = drop_uncommon_columns(train_df, test_df)

test_df

In [ ]:
train_df

In [ ]:
adhd

In [ ]:
sex

# PREDICTING ADHD

In [ ]:
y = adhd
X = train_df.drop(columns=['participant_id'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35)

In [ ]:
import joblib
def train_test_knn(X_train, X_test, y_train, y_test, k):
    '''
    Fit a k Nearest Neighbors classifier to the training data X_train, y_train.
    Return the accuracy of resulting predictions on the test data.
    '''
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X_train, y_train)
    class_label_predictions= model.predict(X_test)
    acc_score = accuracy_score(y_test, class_label_predictions)
    return model, acc_score

In [ ]:
k_values = [10,100,500]
acc1 = []

for k in k_values:
    a_model, score = train_test_knn(X_train, X_test, y_train, y_test, k)
    joblib.dump(a_model, 'knn_adhd_predict.pkl')
    print('k=' + str(k) + ', accuracy score: ' + str(score))
    acc1.append(float(score))

# PREDICTING SEX

In [ ]:
y = sex
X = train_df.drop(columns=['participant_id'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
k_values = [10,100,500]
acc1 = []

for k in k_values:
    s_model, score = train_test_knn(X_train, X_test, y_train, y_test, k)
    joblib.dump(s_model, 'knn_sex_predict.pkl')
    print('k=' + str(k) + ', accuracy score: ' + str(score))
    acc1.append(float(score))

# NOW FOR THE TESTING ON THE TEST DATA

In [ ]:
test_df

In [ ]:
test_df['ADHD_Outcome'] = ""
test_df

In [ ]:
X = test_df.drop(columns=['participant_id', 'ADHD_Outcome'], axis=1)
test_df['ADHD_Outcome'] = a_model.predict(X)
test_df['ADHD_Outcome']

In [ ]:
test_df['Sex_F'] = ""
test_df

In [ ]:
X = test_df.drop(columns=['participant_id', 'Sex_F', 'ADHD_Outcome'], axis=1)
test_df['Sex_F'] = s_model.predict(X)
test_df['Sex_F']

In [ ]:
df = test_df[['participant_id', 'ADHD_Outcome', 'Sex_F']]
df

In [ ]:
print(os.getcwd())
os.chdir('../')
print(os.getcwd())

In [ ]:
filename = 'knn2_submission.csv'
df.to_csv(filename, index=False)